In [1]:
FileUtils.cd '../../'
require './config/boot'
APP_PATH  = File.expand_path('config/application') unless defined?(APP_PATH)
require APP_PATH
Rails.application.require_environment!

true

In [2]:
require "#{Rails.root}/app/models/subject"
require "#{Rails.root}/app/models/year_datum"
require "#{Rails.root}/app/models/faculty"
require "#{Rails.root}/app/models/subject_score"
require "#{Rails.root}/app/models/summarized_subject"

true

In [3]:
require 'open-uri'
require 'nokogiri'
require 'mechanize'
require 'robotex'

false

In [42]:
# robotex = Robotex.new
# p "robots: #{robotex.allowed?(url)}"hash = Hash.new(0)

{}

## 設定

In [198]:
subjects = SummarizedSubject.limit(100)

#<ActiveRecord::Relation [#<SummarizedSubject id: 1, name: "神学-1", code: "0101001-001", number_of_students: nil, A: 0.0, B: 0.0, C: 0.0, D: 0.0, F: 0.0, other: 0.0, mean_score: 0.0, weighted_score: nil, faculty_id: 1, subject_id: 12495, url: "https://syllabus.doshisha.ac.jp/html/2017/0101/101...", term: 0, place: 0, credit: 2, teacher_id: 29, teacher_name: "森山 央朗">, #<SummarizedSubject id: 2, name: "神学-2", code: "0101001-002", number_of_students: nil, A: 0.0, B: 0.0, C: 0.0, D: 0.0, F: 0.0, other: 0.0, mean_score: 0.0, weighted_score: nil, faculty_id: 1, subject_id: 12496, url: "https://syllabus.doshisha.ac.jp/html/2017/0101/101...", term: 0, place: 0, credit: 2, teacher_id: 1895, teacher_name: "木谷 佳楠">, #<SummarizedSubject id: 3, name: "神学-3", code: "0101001-003", number_of_students: nil, A: 0.0, B: 0.0, C: 0.0, D: 0.0, F: 0.0, other: 0.0, mean_score: 0.0, weighted_score: nil, faculty_id: 1, subject_id: 12497, url: "https://syllabus.doshisha.ac.jp/html/2017/0101/101...", term: 0, plac

## スクレイピング
タグ（項目？）ごとにtable, textのハッシュを作成

In [15]:
hash = Hash.new(0)

{}

In [186]:
subject = subjects[16]

#<SummarizedSubject id: 17, name: "聖書学演習15", code: "0101065-000", number_of_students: 14, A: 71.4, B: 7.1, C: 0.0, D: 0.0, F: 21.4, other: 0.0, mean_score: 3.1, weighted_score: 3.1, faculty_id: 1, subject_id: 15, url: "https://syllabus.doshisha.ac.jp/html/2017/0101/101...", term: 1, place: 0, credit: 2, teacher_id: 3, teacher_name: "村山 盛葦">

In [187]:
url = subject.url
agent = Mechanize.new
page = agent.get(url)
data = page.css('body > div > table:nth-child(2) > tbody > tr:nth-child(4) > td')
children=data.children
table_hash = {}
text_hash = {}
tag = 'init'
children.each do |t|
  text = t.text.strip
  if t.node_name == 'table'
    table_hash[tag] = t
  elsif text.present?
    if ret = text.match(/^＜(.+)＞$/)
      p "tag #{tag = ret[1].split('/')[0]}"
    else
      text_hash[tag] = text
    end
  end
end
table_hash["成績評価基準"].css('tbody > tr').each do |tr|
  record_tag = tr.css('td')[0].text.strip
  p record_tag
  hash[record_tag] += 1
end

"tag 概要"
"tag 到達目標"
"tag 授業計画"
"tag 成績評価基準"
"tag テキスト"
"tag 参考文献"
"tag 備考"
"平常点（出席，クラス参加，グループ作業の成果等） "
"期末レポート試験 "
"クラスでの発表など "


0

In [188]:
p subject.subject_id

15


15

In [191]:
table_hash["成績評価基準"].css('tbody > tr').each do |tr|
  record_tag = tr.css('td')[0].text.gsub(/(\xc2\xa0)+/, "").strip
  record_tag = "平常点" if record_tag.include?("平常点") # 平常点系は平常点でまとめる
  record_val = tr.css('td')[1].text.gsub(/(\xc2\xa0)+/, "").strip.gsub("%","")
  p "#{subject.subject_id}, 成績, #{record_tag}, #{record_val}"
end

"15, 成績, 平常点, 30"
"15, 成績, 期末レポート試験, 40"
"15, 成績, クラスでの発表など, 30"


0

In [197]:
["テキスト", "参考文献"].each do |cat|
  if table_hash[cat].present?
    table_hash[cat].css('tbody > tr').each do |c|
      keyword = (c.text.match(/『(.+)』/) || [])[-1]
      if keyword
        p "#{subject.subject_id}, #{cat}, , #{keyword}"
      end
    end
  end
end

"15, テキスト, , 問いかけるイエス－福音書をどう読み解くか－"
"15, 参考文献, , 聖書の風土・歴史・社会"
"15, 参考文献, , イエス研究史"


["テキスト", "参考文献"]

In [202]:
subjects = SummarizedSubject.find(SummarizedSubject.pluck(:id).shuffle[0..100])

[#<SummarizedSubject id: 9887, name: "アナリティカル・リ-ディング1-7", code: "6501023-007", number_of_students: 38, A: 22.975, B: 35.325, C: 23.25, D: 10.325, F: 8.075, other: 0.0, mean_score: 2.55, weighted_score: 2.72727, faculty_id: 18, subject_id: 7275, url: "https://syllabus.doshisha.ac.jp/html/2017/6501/165...", term: 0, place: 1, credit: 1, teacher_id: 1497, teacher_name: "山路 順子">, #<SummarizedSubject id: 11014, name: "ドイツ語入門II-951", code: "6502012-951", number_of_students: nil, A: 0.0, B: 0.0, C: 0.0, D: 0.0, F: 0.0, other: 0.0, mean_score: 0.0, weighted_score: nil, faculty_id: 18, subject_id: 15564, url: "https://syllabus.doshisha.ac.jp/html/2017/6502/165...", term: 0, place: 0, credit: 2, teacher_id: 2999, teacher_name: "湯浅 博章">, #<SummarizedSubject id: 9808, name: "英語講読2-534", code: "6501022-534", number_of_students: nil, A: 0.0, B: 0.0, C: 0.0, D: 0.0, F: 0.0, other: 0.0, mean_score: 0.0, weighted_score: nil, faculty_id: 18, subject_id: 14953, url: "https://syllabus.doshisha.ac.jp/html/

In [203]:
subjects.each do |subject|
  url = subject.url
  agent = Mechanize.new
  page = agent.get(url)
  data = page.css('body > div > table:nth-child(2) > tbody > tr:nth-child(4) > td')
  children=data.children
  table_hash = {}
  text_hash = {}
  tag = 'init'
  children.each do |t|
    text = t.text.strip
    if t.node_name == 'table'
      table_hash[tag] = t
    elsif text.present?
      if ret = text.match(/^＜(.+)＞$/)
        p "tag #{tag = ret[1].split('/')[0]}"
      else
        text_hash[tag] = text
      end
    end
  end
  table_hash["成績評価基準"].css('tbody > tr').each do |tr|
    record_tag = tr.css('td')[0].text.strip
    p record_tag
    hash[record_tag] += 1
  end
  sleep(1)
end

"tag 概要"
"tag 到達目標"
"tag 授業計画"
"tag 成績評価基準"
"tag テキスト"
"平常点(出席，クラス参加，グループ作業の成果等) "
"提出物 "
"小テスト "
"tag 概要"
"tag 到達目標"
"tag 授業計画"
"tag 成績評価基準"
"tag テキスト"
"tag 参照ＵＲＬ"
"tag 備考"
"平常点(出席，クラス参加，小テストの成績等) "
"期末試験 "
"特記事項 "
"tag 概要"
"tag 到達目標"
"tag 授業計画"
"tag 成績評価基準"
"tag テキスト"
"平常点(出席，クラスでの発表，授業態度等) "
"中間試験，小テスト等 "
"期末レポート試験，提出物等 "
"tag 概要"
"tag 到達目標"
"tag 授業計画"
"tag 成績評価基準"
"tag テキスト"
"tag 備考"
"Contributions to the class including presentations, class discussions, attendance "
"Examination "
"tag 概要"
"tag 到達目標"
"tag 授業計画"
"tag 成績評価基準"
"tag テキスト"
"tag 参考文献"
"事前・事後授業への出席・貢献 "
"成果報告（事後授業での発表） "
"成果報告書（最終レポート） "
"現地担当者による評価 "
"tag 概要"
"tag 到達目標"
"tag 授業計画"
"tag 成績評価基準"
"tag 参考文献"
"平常点(出席，クラス参加，グループ作業の成果等) "
"中間レポート試験 "
"期末レポート試験・論文 "
"tag 概要"
"tag 到達目標"
"tag 授業計画"
"tag 成績評価基準"
"tag テキスト"
"tag 備考"
"平常点 "
"小テスト "
"中間試験 "
"期末試験 "
"tag 概要"
"tag 到達目標"
"tag 授業計画"
"tag 成績評価基準"
"tag テキスト"
"tag 参考文献"
"平常点(出席，クラス参加，グループ作業の成果等) "
"期末試験 "
"tag 概要"
"tag 到達目標"
"tag 授業計画"
"tag 成績評価基準"
"tag テキスト"
"tag 備考"
"期末筆記

[#<SummarizedSubject id: 9887, name: "アナリティカル・リ-ディング1-7", code: "6501023-007", number_of_students: 38, A: 22.975, B: 35.325, C: 23.25, D: 10.325, F: 8.075, other: 0.0, mean_score: 2.55, weighted_score: 2.72727, faculty_id: 18, subject_id: 7275, url: "https://syllabus.doshisha.ac.jp/html/2017/6501/165...", term: 0, place: 1, credit: 1, teacher_id: 1497, teacher_name: "山路 順子">, #<SummarizedSubject id: 11014, name: "ドイツ語入門II-951", code: "6502012-951", number_of_students: nil, A: 0.0, B: 0.0, C: 0.0, D: 0.0, F: 0.0, other: 0.0, mean_score: 0.0, weighted_score: nil, faculty_id: 18, subject_id: 15564, url: "https://syllabus.doshisha.ac.jp/html/2017/6502/165...", term: 0, place: 0, credit: 2, teacher_id: 2999, teacher_name: "湯浅 博章">, #<SummarizedSubject id: 9808, name: "英語講読2-534", code: "6501022-534", number_of_students: nil, A: 0.0, B: 0.0, C: 0.0, D: 0.0, F: 0.0, other: 0.0, mean_score: 0.0, weighted_score: nil, faculty_id: 18, subject_id: 14953, url: "https://syllabus.doshisha.ac.jp/html/

In [211]:
new_hash = Hash.new(0)
hash.sort_by {|k, v| -v }.each do |h|
  if h[0].include?("平常点")
    new_hash["平常点"] += h[1].to_i
  else
    new_hash[h[0]] += h[1].to_i
  end
end

[["期末試験 ", 37], ["平常点(出席，クラス参加，グループ作業の成果等) ", 35], ["平常点（出席，クラス参加，グループ作業の成果等） ", 28], ["平常点 ", 28], ["クラスでの発表など ", 22], ["期末レポート試験・論文 ", 21], ["小レポート ", 18], ["期末レポート試験 ", 16], ["中間レポート ", 15], ["期末筆記試験 ", 13], ["小テスト ", 13], ["クラスへの貢献度 ", 11], ["クラスで発表など ", 11], ["期末レポート ", 11], ["提出物 ", 11], ["特記事項 ", 11], ["平常点（出席，意見・感想） ", 8], ["平常点(出席等) ", 6], ["平常点（出席） ", 6], ["その他（具体的に） ", 4], ["平常点(出席，クラス参加等) ", 3], ["平常点（出席，クラス参加，発表，グループ作業の成果等） ", 3], ["期末テスト ", 3], [" ", 3], ["【文法】期末試験 ", 2], ["【実習】期末試験 ", 2], ["平常点（出席，授業態度など） ", 2], ["【実習】小テスト ", 2], ["【実習】平常点(出席，クラス参加，グループ作業の成果等) ", 2], ["【文法】その他（具体的に） ", 2], ["Contributions to the class including presentations, class discussions, attendance ", 2], ["総合評価 ", 2], ["【文法】小テスト ", 2], ["【文法】平常点(出席，クラス参加，グループ作業の成果等) ", 2], ["提出レポート ", 2], ["中間テスト ", 2], ["中間レポート試験 ", 2], ["平常点(出席，感想) ", 2], ["中間試験 ", 2], ["平常点(出席，意見・感想) ", 2], ["中間評価 ", 2], ["オンラインクイズ ", 2], ["【実習】その他（具体的に） ", 2], ["平常点（出席， 中間レポート等） ", 1], ["中間レポート試験および期末レポート試験 ", 1], ["課題 ", 1],

In [212]:
new_hash.sort_by {|k, v| -v }.each do |h|
  p h
end

["平常点", 139]
["期末試験 ", 37]
["クラスでの発表など ", 22]
["期末レポート試験・論文 ", 21]
["小レポート ", 18]
["期末レポート試験 ", 16]
["中間レポート ", 15]
["期末筆記試験 ", 13]
["小テスト ", 13]
["クラスへの貢献度 ", 11]
["クラスで発表など ", 11]
["期末レポート ", 11]
["特記事項 ", 11]
["提出物 ", 11]
["その他（具体的に） ", 4]
[" ", 3]
["期末テスト ", 3]
["【文法】期末試験 ", 2]
["【実習】期末試験 ", 2]
["【実習】小テスト ", 2]
["【文法】その他（具体的に） ", 2]
["Contributions to the class including presentations, class discussions, attendance ", 2]
["総合評価 ", 2]
["【文法】小テスト ", 2]
["提出レポート ", 2]
["中間テスト ", 2]
["中間レポート試験 ", 2]
["中間試験 ", 2]
["中間評価 ", 2]
["オンラインクイズ ", 2]
["【実習】その他（具体的に） ", 2]
["レポート課題 ", 1]
["課題 ", 1]
["CASEC ", 1]
["第１回目の実験から最終回まで，それぞれの実験回数に応じて評価 ", 1]
["授業内評価 ", 1]
["技術点 ", 1]
["取組点 ", 1]
["クラス発表 ", 1]
["論文 ", 1]
["課題点 ", 1]
["学期末試験 ", 1]
["Quizzes ", 1]
["Book report ", 1]
["実力試験（TOEFL） ", 1]
["Written Assignments ", 1]
["Group and Individual Presentations ", 1]
["実験レポート，製作実験，実験ノート ", 1]
["クラスで発表（グループ発表とミニスピーチ） ", 1]
["授業内の課題レポート ", 1]
["その他 ", 1]
["プレゼンテーション ", 1]
["Attendance and participation

[["平常点", 139], ["期末試験 ", 37], ["クラスでの発表など ", 22], ["期末レポート試験・論文 ", 21], ["小レポート ", 18], ["期末レポート試験 ", 16], ["中間レポート ", 15], ["期末筆記試験 ", 13], ["小テスト ", 13], ["クラスへの貢献度 ", 11], ["クラスで発表など ", 11], ["期末レポート ", 11], ["特記事項 ", 11], ["提出物 ", 11], ["その他（具体的に） ", 4], [" ", 3], ["期末テスト ", 3], ["【文法】期末試験 ", 2], ["【実習】期末試験 ", 2], ["【実習】小テスト ", 2], ["【文法】その他（具体的に） ", 2], ["Contributions to the class including presentations, class discussions, attendance ", 2], ["総合評価 ", 2], ["【文法】小テスト ", 2], ["提出レポート ", 2], ["中間テスト ", 2], ["中間レポート試験 ", 2], ["中間試験 ", 2], ["中間評価 ", 2], ["オンラインクイズ ", 2], ["【実習】その他（具体的に） ", 2], ["レポート課題 ", 1], ["課題 ", 1], ["CASEC ", 1], ["第１回目の実験から最終回まで，それぞれの実験回数に応じて評価 ", 1], ["授業内評価 ", 1], ["技術点 ", 1], ["取組点 ", 1], ["クラス発表 ", 1], ["論文 ", 1], ["課題点 ", 1], ["学期末試験 ", 1], ["Quizzes ", 1], ["Book report ", 1], ["実力試験（TOEFL） ", 1], ["Written Assignments ", 1], ["Group and Individual Presentations ", 1], ["実験レポート，製作実験，実験ノート ", 1], ["クラスで発表（グループ発表とミニスピーチ） ", 1], ["授業内の課題レポート ", 1], ["その他 ", 

In [19]:
table_hash

NameError: undefined local variable or method `table_hash' for main:Object

## テキストから書籍名を取ってくる

In [64]:
t=table_hash["参考文献"].text

"\n\n\n\n\n\nハイデッガー \n『存在と時間（I, II, III）』原佑・渡邉二郎訳 (中公クラシックス、2003)\n\n\n\n\n\n \n\n\n\n\n\nレヴィナス \n『全体性と無限（上・下）』熊野純彦訳 (岩波文庫、2006)\n\n\n\n\n\n \n\n\n"

In [66]:
keyword = t.match(/『(.+)』/)[1]

"存在と時間（I, II, III）"

# Amazon URL
検索のキーワードを指定し，検索結果画面のリンクを作成する  
root: "https://www.amazon.co.jp/s/ref=as_li_ss_tl?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&url=search-alias=stripbooks&field-keywords=#{keyword}&rh=n:465392,k:#{keyword}&linkCode=ll2&tag=johshisha-22"

In [67]:
#keyword = "ゼロから始める"
url = "https://www.amazon.co.jp/s/ref=as_li_ss_tl?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&url=search-alias=stripbooks&field-keywords=#{keyword}&rh=n:465392,k:#{keyword}&linkCode=ll2&tag=johshisha-22"

"https://www.amazon.co.jp/s/ref=as_li_ss_tl?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&url=search-alias=stripbooks&field-keywords=存在と時間（I, II, III）&rh=n:465392,k:存在と時間（I, II, III）&linkCode=ll2&tag=johshisha-22"